In [5]:
import cv2

In [6]:
def getFromFile(file,info):
    cap = cv2.VideoCapture(file)
    ret, frame = cap.read()
    cap.release()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    if(info):
        print(f'The shape of the image is {frame_rgb.shape}')
    return frame_rgb

In [7]:
def MaskImg(file,corners):
    image = getFromFile(file,0)
    mask = np.zeros(image.shape[:2], dtype=np.uint8)#CHIEDI
    mask_corners = np.array(corners)
    cv2.drawContours(mask, [mask_corners], -1, 255, -1)
#     plt.imshow(mask, cmap='gray')
    masked_image = cv2.bitwise_and(image,image,mask=mask)
    plt.imshow(masked_image)
    return masked_image

In [8]:
def getLimits(b,g,r,rangeWidth):
    arr = [[[b,g,r]]]
    color = np.uint8(arr)
    hsvColor = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
    lowerLimit = [hsvColor[0][0][0]-rangeWidth,40,40]
    upperLimit = [hsvColor[0][0][0]+rangeWidth,255,255]
    return [lowerLimit,upperLimit]